In [6]:
%load_ext autoreload
%autoreload 2

import argparse
import os
from pathlib import Path

import numpy as np
import pandas as pd
import torch
import yaml
from torch.optim import Adam
from torch.utils.data import DataLoader
from tqdm import tqdm

import wandb
from src import BertClassifier, train_utils, utils
from src import datasets as data_utils

device = utils.get_device()

config = utils.load_config(
    "model_params/bert_classifier.yaml", epochs=10, num_training_examples=-1
)

# Create datasets
train_dataset = data_utils.create_train_sst2(
    device,
    num_samples=config["num_training_examples"],
    tokenizer_name=config["bert_model_name"],
    max_seq_len=config["max_sequence_length"],
)

test_dataset = data_utils.create_test_sst2(
    device,
    tokenizer_name=config["bert_model_name"],
    max_seq_len=config["max_sequence_length"],
)
test_dataloader = DataLoader(test_dataset, shuffle=False, batch_size=1)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 872/872 [00:00<00:00, 10499.25it/s]


In [ ]:
full_model, fdf, full_test_loss, full_test_acc = train_utils.train_bert_model(
    train_dataset, test_dataset, config
)

Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertModel: ['vocab_transform.bias', 'vocab_layer_norm.weight', 'vocab_projector.weight', 'vocab_layer_norm.bias', 'vocab_projector.bias', 'vocab_transform.weight']
- This IS expected if you are initializing DistilBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


 63%|██████████████████████████████████████████████████████████████████████▎                                         | 2645/4210 [03:44<01:11, 22.03batch/s]

In [ ]:
torch.save(full_model.classifier.state_dict(), 'bert-classifier-10epoch-fulldata.pt')

In [ ]:
fdf.to_csv('bert-classifier-10epoch-fulldata-loss.csv', index=False)